In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from keras.callbacks import TensorBoard
from time import time

In [ ]:
data = pd.read_csv("/content/Sentiment.csv")
data.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,I_Am_Kenzi,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,PeacefulQuest,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,PussssyCroook,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,MattFromTexas31,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,sharonDay5,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [ ]:
print(data.groupby('sentiment').nunique())

             id  candidate  ...  tweet_location  user_timezone
sentiment                   ...                               
Negative   8493         11  ...            2926             87
Neutral    3142         11  ...            1419             70
Positive   2236         11  ...            1020             58

[3 rows x 20 columns]


In [ ]:
# Keeping only the neccessary columns
data = data[['text','sentiment']]
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [ ]:
print(data['text'][0])
print(data['text'][1])
print(data['text'][2])

RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate
RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…
RT @TJMShow: No mention of Tamir Rice and the #GOPDebate was held in Cleveland? Wow.


In [ ]:
data['text'] = data['text'].apply(lambda x: x.lower()) # all string to lowercase Read more about lambda() "https://realpython.com/python-lambda/"
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x))) # using regular expression preprocess the text by removing everything that is not [a-zA-z0-9\s]

In [ ]:
print(data['text'][0])
print(data['text'][1])
print(data['text'][2])

rt nancyleegrahn how did everyone feel about the climate change question last night exactly gopdebate
rt scottwalker didnt catch the full gopdebate last night here are some of scotts best lines in 90 seconds walker16 httptcozsff
rt tjmshow no mention of tamir rice and the gopdebate was held in cleveland wow


In [ ]:
for idx, row in data.iterrows(): # Iterate over DataFrame rows as (index, Series) pairs.
    row[0] = row[0].replace('rt', '') # replace/ remove 'rt' in the start of the text

In [ ]:
print(data['text'][0])
print(data['text'][1])
print(data['text'][2])

 nancyleegrahn how did everyone feel about the climate change question last night exactly gopdebate
 scottwalker didnt catch the full gopdebate last night here are some of scotts best lines in 90 seconds walker16 httptcozsff
 tjmshow no mention of tamir rice and the gopdebate was held in cleveland wow


In [ ]:
max_fatures = 2000

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)

In [ ]:
# Transforms each text in texts to a sequence of integers.
# Only top num_words-1 most frequent words will be taken into account. Only words known by the tokenizer will be taken into account.

# Transforms each text in texts to a sequence of integers. 
# So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary.
X = tokenizer.texts_to_sequences(data['text'].values)

In [ ]:
# tokenizer.get_config()
# Returns the tokenizer configuration as Python dictionary.
print(tokenizer.get_config().keys())

# word_counts: A dictionary of words and their counts.
print("\ntokenizer.word_counts") 
print(tokenizer.word_counts) 

# document_count:An integer count of the total number of documents that were used to fit the Tokenizer.
print("\ntokenizer.document_count")
print(tokenizer.document_count) 

# word_index: A dictionary of words and their uniquely assigned integers.
print("\ntokenizer.word_index")
print(tokenizer.word_index)

# word_docs: A dictionary of words and how many documents each appeared in.
print("\ntokenizer.word_docs")
print(tokenizer.word_docs)



print('\nLen() of X:', len(X))
print('\n', X[:2])

dict_keys(['num_words', 'filters', 'lower', 'split', 'char_level', 'oov_token', 'document_count', 'word_counts', 'word_docs', 'index_docs', 'index_word', 'word_index'])

tokenizer.word_counts
OrderedDict([('nancyleegrahn', 6), ('how', 623), ('did', 434), ('everyone', 91), ('feel', 64), ('about', 1146), ('the', 8903), ('climate', 72), ('change', 84), ('question', 350), ('last', 888), ('night', 632), ('exactly', 22), ('gopdebate', 9014), ('scottwalker', 88), ('didnt', 270), ('catch', 9), ('full', 45), ('here', 145), ('are', 820), ('some', 167), ('of', 3376), ('scotts', 2), ('best', 185), ('lines', 14), ('in', 1986), ('90', 12), ('seconds', 15), ('walker16', 28), ('httptcozsff', 1), ('tjmshow', 1), ('no', 528), ('mention', 56), ('tamir', 2), ('rice', 2), ('and', 2896), ('was', 1149), ('held', 10), ('cleveland', 37), ('wow', 53), ('robgeorge', 1), ('that', 1445), ('carly', 118), ('fiorina', 140), ('is', 3356), ('trending', 2), ('hours', 34), ('after', 348), ('her', 192), ('debate', 1051), 

In [ ]:
# Check the sequence of the text, Do we need to Pad ??
for i in range(4):
  print(X[i])
  print('len=', len(X[i]))

[52, 78, 338, 449, 22, 2, 413, 361, 95, 29, 51, 1023, 1]
len= 13
[348, 123, 1937, 2, 588, 1, 29, 51, 226, 35, 195, 5, 172, 1400, 10, 1561, 1336, 833]
len= 18
[62, 491, 5, 8, 2, 1, 21, 1781, 10, 685, 512]
len= 11
[17, 276, 235, 6, 723, 96, 164, 24, 130, 5, 2, 176, 10, 1, 211, 773, 16]
len= 17


In [ ]:
X = pad_sequences(X) # Pads sequences to the same length.
print('X.shape = ', X.shape)

X.shape =  (13871, 28)


In [ ]:
# Check the sequence after padding, Which padding pre or post??
for i in range(4):
  print(X[i])
  print('len=', len(X[i]))

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0   52   78  338  449   22    2  413  361   95   29   51 1023    1]
len= 28
[   0    0    0    0    0    0    0    0    0    0  348  123 1937    2
  588    1   29   51  226   35  195    5  172 1400   10 1561 1336  833]
len= 28
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0   62  491    5    8    2    1   21 1781   10  685  512]
len= 28
[  0   0   0   0   0   0   0   0   0   0   0  17 276 235   6 723  96 164
  24 130   5   2 176  10   1 211 773  16]
len= 28


In [ ]:
embed_dim = 128
lstm_out = 196

def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    model.summary()
    return model

In [ ]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [ ]:
model = createmodel()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 1)

Epoch 1/7
291/291 [==============================] - 41s 129ms/step - loss: 0.8944 - accuracy: 0.6194
Epoch 2/7
291/291 [==============================] - 38s 132ms/step - loss: 0.6851 - accuracy: 0.7044
Epoch 3/7
291/291 [==============================] - 37s 129ms/step - loss: 0.5956 - accuracy: 0.7527
Epoch 4/7
291/291 [==============================] - 38s 132ms/step - loss: 0.5478 - accuracy: 0.7771
Epoch 5/7
291/291 [==============================] - 37s 128ms/step - loss: 0.4928 - accuracy: 0.7988
Epoch 6/7
291/291 [==============================] - 38s 129ms/step - loss: 0.4581 - accuracy: 0.8206
Epoch 7/7
291/291 [==============================] - 39s 133ms/step - loss: 0.4142 - accuracy: 0.8341


# 1. Train and save the model and use the saved model to predict on new text data (ex, **“A lot of good things are happening. We are respected again throughout the world, and that's a great thing.**@realDonaldTrump”)

## Save the model

In [ ]:
# Save the model
model.save('sentiment_model.h5')

# Loading the model
saved_model = load_model('sentiment_model.h5')

## Model evaluation

In [ ]:
# Model evaluation
scores = saved_model.evaluate(X_test, Y_test, verbose=0)
print("Test Accuracy for sentiment data model: %.2f%%" % (scores[1]*100))

Test Accuracy for spam data model: 65.49%


In [ ]:
# Predict the sentiment for new data
tweet = ['A lot of good things are happening. We are respected again throughout the world, and thats a great thing.@realDonaldTrump']

# vectorizing the tweet by the pre-fitted tokenizer instance
tweet = tokenizer.texts_to_sequences(tweet)

# padding the tweet to have exactly the same shape as `embedding_2` input
tweet = pad_sequences(tweet, maxlen=28, dtype='int32', value=0)
print(tweet)

sentiment = model.predict(tweet, batch_size=1, verbose=2)[0]
print(sentiment)

[[  0   0   0   0   0   0   0   0   0   0   0   7 438   5 144 289  35  30
   35 371   2 346   8 260   7 151 263  23]]
1/1 - 1s
[0.09895065 0.0333237  0.8677257 ]


## Sentiment prediction of the given tweet

In [ ]:
print("Final sentiment prediction for the tweet provided:")
if (np.argmax(sentiment) == 0):
    print("Negative")
elif (np.argmax(sentiment) == 1):
    print("Positive")
else:
    print("Neutral")

Final sentiment prediction for the tweet provided:
Neutral


# 2. Apply the code on spam data set available in the source code (text classification on the spam.csv data set)

In [ ]:
spam_data = pd.read_csv('/content/spam.csv',encoding='latin-1')
# Keeping only the neccessary columns
spam_data = spam_data[['v2','v1']]

spam_data['v2'] = spam_data['v2'].apply(lambda x: x.lower())
spam_data['v2'] = spam_data['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in spam_data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(spam_data['v2'].values)
X = tokenizer.texts_to_sequences(spam_data['v2'].values)
print(X)
X = pad_sequences(X)
print(X)
embed_dim = 128

from sklearn.preprocessing import LabelEncoder
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(spam_data['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

[[47, 445, 800, 717, 684, 64, 9, 1250, 90, 119, 361, 1251, 149, 1252, 67, 58, 137], [48, 316, 1403, 446, 6, 1830], [50, 466, 9, 22, 4, 755, 903, 1, 180, 1595, 1136, 630, 1831, 266, 71, 1595, 1, 1832, 1, 317, 466, 79, 385], [6, 233, 143, 24, 362, 6, 161, 144, 60, 143], [954, 2, 43, 97, 72, 467, 1, 904, 72, 1833, 206, 111, 447], [849, 120, 67, 1596, 41, 100, 186, 602, 21, 7, 39, 349, 86, 363, 56, 109, 386, 3, 42, 12, 15, 84, 1834, 48, 374, 1041, 1, 70, 306, 1], [207, 11, 603, 8, 25, 56, 1, 364, 35, 10, 108, 685, 10, 56], [73, 220, 13, 1137, 1404, 1404, 1835, 112, 100, 578, 73, 13, 955, 905, 12, 51, 1597, 956, 468, 1, 1042, 13, 221, 955, 905], [686, 73, 4, 801, 415, 222, 3, 17, 100, 416, 1, 1043, 140, 957, 1, 115, 16, 115, 387, 508, 718, 552, 64], [129, 13, 92, 1044, 802, 27, 124, 6, 85, 1138, 1, 469, 1, 5, 307, 529, 850, 35, 333, 12, 50, 16, 5, 92, 469, 1253, 50, 18], [23, 223, 32, 80, 213, 7, 2, 43, 69, 1, 269, 77, 40, 281, 1139, 214, 181, 169, 417, 91], [1836, 1, 180, 174, 46, 318, 1, 

In [ ]:
batch_size = 32
spam_model = createmodel()
tensorborad = TensorBoard(log_dir="logs/{}".format(time()))

spam_model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2,  callbacks=[tensorborad])

Epoch 1/5
117/117 - 73s - loss: 0.2186 - accuracy: 0.9282
Epoch 2/5
117/117 - 66s - loss: 0.0501 - accuracy: 0.9845
Epoch 3/5
117/117 - 65s - loss: 0.0255 - accuracy: 0.9928
Epoch 4/5
117/117 - 67s - loss: 0.0208 - accuracy: 0.9954
Epoch 5/5
117/117 - 65s - loss: 0.0130 - accuracy: 0.9971


# Model Evaluation for spam data

In [ ]:
# Save the model
spam_model.save('spam_model.h5')
# Loading the model
saved_spam_model = load_model('spam_model.h5')

## Spam data model evaluation

In [ ]:
# Model evaluation
scores = saved_spam_model.evaluate(X_test, Y_test, verbose=0)
print("Test Accuracy for spam data model: %.2f%%" % (scores[1]*100))

Test Accuracy for spam data model: 98.37%
